In [101]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr
import math
from scipy.stats import wasserstein_distance

In [102]:
Predicted = pd.read_csv("./SpatialDWLS/Outputs/output_Sim13.csv")
Target=pd.read_csv("./Data/Sim13/target.csv")

In [103]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
Predicted.drop(columns="cell_ID", inplace=True)
#Predicted.drop(columns="prediction.score.max",inplace=True)


In [104]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [105]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [106]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [107]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [108]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [109]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [110]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [111]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [112]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

In [113]:
np.mean(np.nan_to_num(d))

0.8081361060814958

In [114]:
d

array([0.91576508, 0.24550005, 0.86511059, 0.88311656, 0.89108735,
       0.91968386, 0.86383506, 0.75130675, 0.88040784, 0.86554793])

## Jenson Shannon Divergence

In [115]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.20446705907883522

## F1 Score, Precision, Recall

In [116]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [117]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [118]:
d

,Precision,Recall,F1-score
0.100,0.947492,0.842802,0.892086
0.200,0.858046,0.801908,0.829028
0.300,0.756402,0.735163,0.745631
0.010,0.965478,0.849455,0.903758
0.001,0.960075,0.849455,0.901384


## Wasserstein Distance

In [119]:
scipy.stats.wasserstein_distance(np.reshape(Target,(Target.shape[0]*Target.shape[1])),np.reshape(Predicted,(Predicted.shape[0]*Predicted.shape[1])))

0.016553612843457938

## KL Divergence

In [120]:
to_be_kept=(Predicted>0)

In [121]:
Predicted=Predicted[to_be_kept]
Target=Target[to_be_kept]

In [122]:
kl=scipy.special.rel_entr(Target,Predicted)

In [123]:
np.mean(kl)

0.01756192682135398